In [1]:
!pip install ortools -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 9.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.26.1 which is incompatible.
tensorflow-metadata 1.13.1 requires absl-py<2.0.0,>=0.9, but you have absl-py 2.1.0 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 5.26.1 which is incompatible.


In [2]:
from ortools.sat.python import cp_model

# OR-Tools approach

In [3]:
#@title Replenishment costs: solving for multiple atms in many days
def solve_multiple_atms(CAPACIDAD, coef, saldos_iniciales, predicciones_demandas):
    n_cajeros = len(saldos_iniciales)
    ndias = len(predicciones_demandas[0])
    for i in range(n_cajeros):
        predicciones_demandas[i].insert(0, 0)
    model = cp_model.CpModel()

    saldo_final = {}
    abastecimiento = {}
    is_ab = {}

    for atm in range(n_cajeros):
        saldo_final[atm] = [model.NewIntVar(0, CAPACIDAD, f'sf[{atm},{i}]') for i in range(ndias + 1)]
        abastecimiento[atm] = [model.NewIntVar(0, CAPACIDAD, f'ab[{atm},{i}]') for i in range(ndias + 1)]
        is_ab[atm] = [model.NewBoolVar(f'is_ab[{atm},{i}]') for i in range(ndias + 1)]

        model.Add(saldo_final[atm][0] == saldos_iniciales[atm])

        for i in range(1, ndias + 1):
            model.Add(saldo_final[atm][i - 1] - saldo_final[atm][i] + abastecimiento[atm][i] == predicciones_demandas[atm][i])

            model.Add(saldo_final[atm][i] >= int(coef * CAPACIDAD))
            model.Add(abastecimiento[atm][i] + saldo_final[atm][i - 1] <= CAPACIDAD)
            model.Add(abastecimiento[atm][i] == 0).OnlyEnforceIf(is_ab[atm][i].Not())
            model.Add(abastecimiento[atm][i] > 0).OnlyEnforceIf(is_ab[atm][i])

    total_abastecimiento = sum(abastecimiento[atm][i] for atm in range(n_cajeros) for i in range(1, ndias + 1))
    model.Minimize(total_abastecimiento)

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        abastecimiento_resultado = {
            atm: [solver.Value(abastecimiento[atm][i]) for i in range(1, ndias + 1)] for atm in range(n_cajeros)
        }
        is_ab_resultado = {
            atm: [solver.Value(is_ab[atm][i]) for i in range(1, ndias + 1)] for atm in range(n_cajeros)
        }
        return abastecimiento_resultado, is_ab_resultado, ndias
    print("No se encontró solución en la optimización de abastecimientos")
    return None, None, None

In [4]:
#@title Transport cost: one bus visits many ATMs

def build_submatrix(atms_to_visit, mat):
    indices = [0] + [atm + 1 for atm in atms_to_visit]
    submatrix = []
    for i in indices:
        row = [mat[i][j] for j in indices]
        submatrix.append(row)
    return submatrix


def minimize_cost(atms_to_visit, cost_matrix):
    n_atms = len(atms_to_visit)
    n = n_atms + 2 # suma inicio y fin

    model = cp_model.CpModel()

    V = [model.NewIntVar(0, n - 1, f'V[{i}]') for i in range(n)]
    # empiza en el 0,0 y termina en 0,0
    model.Add(V[0] == 0)
    model.Add(V[n - 1] == 0)

    for i in range(1, n - 1):
        model.Add(V[i] >= 1)
        model.Add(V[i] <= n_atms)

    model.AddAllDifferent(V[1:n - 1])

    mx = n*max(max(row) for row in cost_matrix)
    cost_vars = []
    matrix_flat = sum(cost_matrix, [])
    size = len(cost_matrix)

    for i in range(n - 1):
        index = model.NewIntVar(0, size * size - 1, f'index[{i}]')
        model.Add(index == V[i] * size + V[i + 1])
        c = model.NewIntVar(0, mx, f'c[{i}]')
        model.AddElement(index, matrix_flat, c)
        cost_vars.append(c)

    model.Minimize(sum(cost_vars))

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        sequence = [solver.Value(V[i]) for i in range(n)]
        route = ['Depósito']
        for idx in sequence[1:n-1]:
            atm_number = atms_to_visit[idx - 1] + 1  # +1 para cajero 1-based
            route.append(f'Cajero {atm_number}')
        route.append('Depósito')
        return route, solver.ObjectiveValue()
    else:
        print('No se encontró solución en la optimización de transporte.')
        return None, None

In [5]:
#@title Main
def solve(CAPACIDAD, coef, saldos_iniciales, predicciones_demandas, cost_matrix_full):
    resultados, dias_abastecimiento, ndias = solve_multiple_atms(CAPACIDAD, coef, saldos_iniciales, predicciones_demandas)
    if not resultados: return
    n_atms = len(saldos_iniciales)
    for atm in range(n_atms):
        print(f"Cajero {atm+1}:")
        print(f"  Abastecimientos: {resultados[atm]}")
        print(f"  Días de abastecimiento: {dias_abastecimiento[atm]}")

    dias_a_abastecer = {}
    for day in range(ndias):
        atms_to_replenish = []
        for atm in range(n_atms):
            if dias_abastecimiento[atm][day] == 1:
                atms_to_replenish.append(atm)
        dias_a_abastecer[day] = atms_to_replenish

    for day in range(ndias):
        atms_to_replenish = dias_a_abastecer[day]
        print(f"\nDía {day + 1}:")
        if atms_to_replenish:
            cost_matrix = build_submatrix(atms_to_replenish, cost_matrix_full)
            route, total_cost = minimize_cost(atms_to_replenish, cost_matrix)
            if route:
                print(f"Costo total de transporte: {total_cost}")
                print(f"Ruta óptima: {route}")
        else:
            print(f"\nNo hay cajeros para abastecer.")

In [6]:
#@title Ejemplo

CAPACIDAD = 100
coef = 0.2
saldos_iniciales = [30, 50]
predicciones_demandas = [
    [20],
    [60]
]

cost_matrix_full = [
    [0,2,5], #0->1->2 = 0 + 3 + 7 + 5 = 15
    [3,0,7], #0->2->1 = 0 + 5 + 7 + 2 = 14
    [5,7,0]
]
solve(CAPACIDAD, coef, saldos_iniciales, predicciones_demandas, cost_matrix_full)

Cajero 1:
  Abastecimientos: [10]
  Días de abastecimiento: [1]
Cajero 2:
  Abastecimientos: [30]
  Días de abastecimiento: [1]

Día 1:
Costo total de transporte: 14.0
Ruta óptima: ['Depósito', 'Cajero 1', 'Cajero 2', 'Depósito']
